In [1]:
import librosa
import numpy as np
from matplotlib import pyplot as plt
from pydub import AudioSegment
from scipy import signal
from sklearn import preprocessing
import os
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa.display
import IPython.display as ipd
import warnings
warnings.simplefilter("ignore", UserWarning)
from tensorflow.keras.callbacks import EarlyStopping
import random

In [ ]:
#import tensorflow_io as tfio

In [3]:
os.getcwd()
os.chdir('C:/Users/hennm/Desktop/')

In [4]:
os.getcwd()

'C:\\Users\\hennm\\Desktop'

In [15]:
pathAudio = 'C:/Users/hennm/Desktop/sample-same/'
files = librosa.util.find_files(pathAudio, ext=['mp3'])
songa = []
songb = []
songc = []
songbad = []
for file in files:
    if "Holizna" in file:
        songa.append(file)
    if "Lounge_it" in file:
        songb.append(file)
    if "Tuesday" in file:
        songc.append(file)
    if "John" in file:
        songbad.append(file)
filesa = sorted(songa, key=lambda x: int(x.split('_')[-1].split(".")[0]))
filesb = sorted(songb, key=lambda x: int(x.split('_')[-1].split(".")[0]))
filesc = sorted(songc, key=lambda x: int(x.split('_')[-1].split(".")[0]))
filesbad = sorted(songbad, key=lambda x: int(x.split('_')[-1].split(".")[0]))
filesa = filesa[:-1]
filesb = filesb[:-1]
filesc = filesc[:-1]
filesbad = filesbad[:-1]

files = filesa+filesb+filesc


['C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_1.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_2.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_3.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_4.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_5.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_6.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_7.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_8.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_9.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_10.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_11.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_12.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_13.mp3',
 'C:\\Users\\hennm\\Desktop\\BadSong\\JohnLopker-SeverelyClear_14.mp3',
 

In [18]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


samples = []
samples_sec_canal = []
samplesbad = []
for file_name in files:
    x, sr = librosa.load(file_name, sr=44100)  
    normalized_sample = NormalizeData(x)
    #resampled_sample = signal.resample(normalized_sample, 44100)
    #resampled_sample = signal.resample(normalized_sample, int(len(normalized_sample) / 10))
    #samples.append(resampled_sample)
    samples.append(normalized_sample)
for a in filesbad:
    b, sr = librosa.load(a, sr=44100)
    normalized_sample_bad = NormalizeData(b)
    #resampled_sample = signal.resample(normalized_sample, 44100)
    #resampled_sample = signal.resample(normalized_sample, int(len(normalized_sample) / 10))
    #samples.append(resampled_sample)
    samplesbad.append(normalized_sample_bad)


In [5]:
#Play Audio File!!

##import IPython.display as ipd
##ipd.Audio(files[2])

In [6]:
#example_x_train = samples[3]
#plt.plot(example_x_train)
#plt.show()

In [7]:
X_train = []
Y_train = []
for i in range(len(samples)):
    Y_train.append(1)
    if i < len(samples)-1:
        x = list(samples[i])
        y = list(samples[i+1])
        z = x + y
        X_train.append(z)
    else:
        x = list(samples[i])
        y = list(samples[0])
        z = x + y
        X_train.append(z)

In [8]:
len(X_train)

166

In [9]:
import random

X_false = []
Y_false = []
shuffled = samples
random.shuffle(shuffled)
for i in range(len(shuffled)):
    Y_false.append(0)
    if i < len(shuffled)-1:
        x = list(shuffled[i])
        y = list(shuffled[i+1])
        z = x + y
        X_false.append(z)   
    else:
        x = list(shuffled[i])
        y = list(shuffled[0])
        z = x + y
        X_false.append(z)


In [10]:
len(X_false)

166

In [11]:
X_train_total = []
Y_train_total = []

X_train_total.append(X_train)
X_train_total.append(X_false)
Y_train_total.append(Y_train)
Y_train_total.append(Y_false)

In [12]:
print(len(Y_false))
#for a in Y_train_total:
#    print(len(a))

166


In [13]:
X_train_total = np.array(X_train_total)
X_train_total.shape

(2, 166, 264600)

In [14]:
Y_train_total = np.array(Y_train_total)
Y_train_total.shape

(2, 166)

In [15]:
X_train_total = X_train_total.reshape(-1, 264600 )
X_train_total.shape

(332, 264600)

In [16]:
Y_train_total = np.array(Y_train_total).reshape(-1, 1)
Y_train_total.shape

(332, 1)

In [18]:
from tensorflow import keras
from keras.layers import LSTM, Dense, Conv1D, MaxPool1D, Flatten, Dropout
from keras.models import Sequential

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, input_shape=(264600, 1))) 
model.add(MaxPool1D(pool_size=2, strides=2))
model.add(Conv1D(filters=32, kernel_size=2)) 
model.add(MaxPool1D(pool_size=2, strides=2))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='sigmoid')) 
model.add(Dense(32))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])  
model.summary()

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/opt/miniconda3/envs/python3/lib/python3.9/site-packages/tensorflow/python/eager/context.py)

In [ ]:
model.evaluate(X_train_total, Y_train_total)

In [ ]:

model.fit(X_train_total, Y_train_total, epochs=2)

# 17 min

In [ ]:
example_x_train = X_train_total[1]
plt.plot(example_x_train)
plt.show()
example_x_train = example_x_train.reshape(1, 26460     , 1)
model.predict(example_x_train)

In [ ]:
example_x_train = X_train_total[-2]
plt.plot(example_x_train)
plt.show()
example_x_train = example_x_train.reshape(1, 26460   , 1)
model.predict(example_x_train)

In [ ]:
samples_sec_ch = np.array(samples_sec_canal)
full_song_sec_ch = []
for sample in samples_sec_ch:
    full_song_sec_ch.extend(sample)
second_channel_data = full_song_sec_ch[4000:6000]
plt.plot(second_channel_data)
plt.show()
second_channel_data = np.array(second_channel_data)
second_channel_data = second_channel_data.reshape(1, 2000)
model.predict(second_channel_data)

In [ ]:
extracted_sample = full_song_sec_ch[4000:6000]
new_start = extracted_sample[999:1999]
new_end = extracted_sample[0:1000]
switched_sample = new_start + new_end
plt.plot(switched_sample)
plt.show()
switched_sample = np.array(switched_sample)
switched_sample = switched_sample.reshape(1, 2000)
model.predict(switched_sample)

In [ ]:
reversed_sample = reversed_data[4000:6000]
plt.plot(reversed_sample)
plt.show()
reversed_sample = np.array(reversed_sample)
reversed_sample = reversed_sample.reshape(1, 2000)
model.predict(reversed_data)

In [ ]:
#filename = 'model_weights.h5'
#model.save(filename)